<a href="https://colab.research.google.com/github/garynico/IndoBERT-Hoax/blob/main/Tf_Idf_%26_SVM_Hoax_News_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
np.random.seed(500)

In [3]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
Encoder = LabelEncoder()
Tfidf_vect = TfidfVectorizer()

In [31]:

# Open Datasets
datasets = pd.read_csv('https://raw.githubusercontent.com/garynico/IndoBERT-Hoax/main/data_resample.csv')
print(datasets["label"].value_counts())

# Text Normalization using
# PySastrawi(Word Stemmer for Bahasa Indonesia)
lower = [stemmer.stem(row.lower()) for row in datasets["narasi"]]
vectors = [word_tokenize(element) for element in lower]
labels = datasets["label"]

# Splitting Datasets for feeding to Machine Learning
Train_X, Test_X, Train_Y, Test_Y = train_test_split(
    vectors, labels, test_size=0.2, stratify=labels)

# Encoder for Data Label
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

# Create Tfidf Vector
Tfidf_vect.fit(["".join(row) for row in lower])

# Applying Tfidf for Training and Testing Features
Train_X_Tfidf = Tfidf_vect.transform([" ".join(row) for row in Train_X])
Test_X_Tfidf = Tfidf_vect.transform([" ".join(row) for row in Test_X])

# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=1, gamma="auto", verbose=True)
SVM.fit(Train_X_Tfidf, Train_Y)  # predict the labels on validation dataset
# Use accuracy_score function to get the accuracy
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Test_Y)*100)

1    3465
0    3465
Name: label, dtype: int64
[LibSVM]SVM Accuracy Score ->  89.68253968253968


In [32]:
print("\nConfusion Matrix of SVM Classifier:\n")
print(confusion_matrix(predictions_SVM, Test_Y))
print("\nClassification Report of SVM Classifier:\n")
print(classification_report(predictions_SVM, Test_Y)) 


Confusion Matrix of SVM Classifier:

[[635  85]
 [ 58 608]]

Classification Report of SVM Classifier:

              precision    recall  f1-score   support

           0       0.92      0.88      0.90       720
           1       0.88      0.91      0.89       666

    accuracy                           0.90      1386
   macro avg       0.90      0.90      0.90      1386
weighted avg       0.90      0.90      0.90      1386

